In [0]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
from torch import nn
from torch import optim

import math

In [0]:
from mathpl

In [0]:
from tqdm import tqdm as tqdm

In [0]:
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [0]:
m = 128
n = 3

w = np.random.randn(n)

X_train = np.random.randn(m, n)
Y_train = (X_train@w).reshape([-1, 1])

X_test = np.random.randn(m, n)
Y_test = (X_test@w).reshape([-1, 1])

In [0]:
train_data = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
test_data = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

In [0]:
X_ts, Y_ts = test_data[:]

X_tr, Y_tr = train_data[:]

In [0]:
def gradient(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False):
    '''
    Compute the gradient of `outputs` with respect to `inputs`
    gradient(x.sum(), x)
    gradient((x * y).sum(), [x, y])
    '''
    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)
    grads = torch.autograd.grad(outputs, inputs, grad_outputs,
                                allow_unused=True,
                                retain_graph=retain_graph,
                                create_graph=create_graph)
    grads = [x if x is not None else torch.zeros_like(y) for x, y in zip(grads, inputs)]
    return torch.cat([x.contiguous().view(-1) for x in grads])


def jacobian(outputs, inputs, create_graph=False):
    '''
    Compute the Jacobian of `outputs` with respect to `inputs`
    jacobian(x, x)
    jacobian(x * y, [x, y])
    jacobian([x * y, x.sqrt()], [x, y])
    '''
    if torch.is_tensor(outputs):
        outputs = [outputs]
    else:
        outputs = list(outputs)

    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)

    jac = []
    for output in outputs:
        output_flat = output.view(-1)
        output_grad = torch.zeros_like(output_flat)
        for i in range(len(output_flat)):
            output_grad[i] = 1
            jac += [gradient(output_flat, inputs, output_grad, True, create_graph)]
            output_grad[i] = 0
    return torch.stack(jac)

def hessian(output, inputs, out=None, allow_unused=False, create_graph=False):
    '''
    Compute the Hessian of `output` with respect to `inputs`
    hessian((x * y).sum(), [x, y])
    '''
    assert output.ndimension() == 0

    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)

    n = sum(p.numel() for p in inputs)
    if out is None:
        out = output.new_zeros(n, n)

    ai = 0
    for i, inp in enumerate(inputs):
        [grad] = torch.autograd.grad(output, inp, create_graph=True, allow_unused=allow_unused)
        grad = torch.zeros_like(inp) if grad is None else grad
        grad = grad.contiguous().view(-1)

        for j in range(inp.numel()):
            if grad[j].requires_grad:
                row = gradient(grad[j], inputs[i:], retain_graph=True, create_graph=create_graph)[j:]
            else:
                row = grad[j].new_zeros(sum(x.numel() for x in inputs[i:]) - j)

            out[ai, ai:].add_(row.type_as(out))  # ai's row
            if ai + 1 < n:
                out[ai + 1:, ai].add_(row[1:].type_as(out))  # ai's column
            del row
            ai += 1
        del grad

    return out

In [0]:
class Neural(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=3, output_dim=1, device='cpu'):
        """
        """
        super(Neural, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.device = device

#         self.body = nn.Sequential(
#             nn.Linear(self.input_dim, self.hidden_dim),
#             nn.LeakyReLU(),
#         )
        
        self.head = nn.Linear(self.input_dim, self.output_dim)
        
        list_of_w = []
        for w in self.parameters():
            list_of_w.append(w.data.view(-1))
        self.D = len(torch.cat(list_of_w))
        self.S = 0.5*self.D*(1+2*math.pi)

        self.to(device)

        
    def predict(self, input):
        """
        Args:
            input: Tensor(batch_size x input_dim) --- the matrix of input data
            
        Returns:
            Tensor(batch_size x output_dim) --- the matrix of output data
        
        """
#         out = self.body(input)
        return self.head(input)
    
    def log_prior_w(self, w):
        """
        """
        sigma = 0.
        w = w.view(-1)
        return -0.5*(sigma**2)*torch.dot(w, w)
      
    def log_priot_all(self):
        temp = 0
        for w in self.parameters():
            temp += self.log_prior_w(w)
        return temp
     
    def loglikelihood(self, batch_x, batch_y):
        """
        """
        beta = 10
        D = len(batch_y)
        return -0.5*beta*torch.sum((self.predict(batch_x) - batch_y)**2)
    
    def margin_likelihood(self, batch_x, batch_y):
        """
        """
        temp = self.log_priot_all() + self.loglikelihood(batch_x, batch_y)
        return temp
    
    def loss(self, batch_x, batch_y):
      out = self.predict(batch_x)
#       return torch.nn.MSELoss()(out, batch_y)
      return torch.mean((out - batch_y)**2)
    

In [0]:
model = Neural(input_dim=n, output_dim=1)
List_of_step = []

In [11]:
alpha = 0.0001



for ep in tqdm(range(1000)):
    generator = DataLoader(train_data, batch_size=4, shuffle=True)
    for batch_x, batch_y in generator:
        # оптимизация энтропии
        model.zero_grad()
        margin_likelihood = -model.margin_likelihood(batch_x, batch_y)

        list_of_w = []
        for w in model.parameters():
            list_of_w.append(w)

        model.S = (model.S+torch.log(torch.det(torch.eye(model.D) - alpha*hessian(margin_likelihood, list_of_w)))).detach()

        # оптимизация параметров сетки     
        model.zero_grad()

        optimazer = optim.SGD(model.parameters(), lr=alpha)

        loss = model.loss(batch_x, batch_y)
        loss.backward()

        #         

        #         for w in model.parameters():
        #           w.data = (w.data - alpha*w.grad).detach()



        optimazer.step()

    List_of_step.append((model.S.item(), model.log_priot_all().item(), model.loglikelihood(X_tr, Y_tr).item(), model.loglikelihood(X_ts, Y_ts).item()))
        
        


100%|██████████| 1000/1000 [01:58<00:00,  8.37it/s]


In [12]:
model.S

tensor(-521.5541)

In [13]:
((model.predict(X_ts) - Y_ts)**2).mean()

tensor(1.4032e-05, grad_fn=<MeanBackward0>)

In [14]:
((model.predict(X_tr) - Y_tr)**2).mean()

tensor(1.3127e-05, grad_fn=<MeanBackward0>)